# Installation

In [ ]:
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install pymysql

Import libraries

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as db
import json

# Extract

Read CSV files

Load Progress

In [2]:
doctors = pd.read_parquet('cleaned/doctors.parquet')
clinics = pd.read_parquet('cleaned/clinics.parquet')
px = pd.DataFrame()
for i in range(8):
    temp = pd.read_parquet(f"cleaned/px_{i+1}.parquet")
    px = pd.concat([px,temp])
appointments = pd.read_parquet('cleaned/appointments.parquet')

In [3]:
del doctors['age']

In [4]:
merge_df = pd.merge(appointments, clinics, on = 'clinicid', how = 'inner')

In [5]:
merge_df = pd.merge(merge_df, doctors, on = 'doctorid', how = 'inner')

In [6]:
merge_df = pd.merge(merge_df, px, on = 'pxid', how = 'inner')

In [7]:
merge_df.rename(columns={'age': 'patientage'}, inplace=True)
merge_df.rename(columns={'gender': 'patientgender'}, inplace=True)

In [8]:
luzon_df = merge_df[merge_df['RegionName'].isin(['National Capital Region (NCR)', 
                                                 'CALABARZON (IV-A)', 
                                                 'Ilocos Region (I)', 
                                                 'Bicol Region (V)', 
                                                 'Central Luzon (III)'])]

visayas_mindanao_df = merge_df[merge_df['RegionName'].isin(['Central Visayas (VII)', 
                                                            'SOCCSKSARGEN (Cotabato Region) (XII)', 
                                                            'Northern Mindanao (X)', 
                                                            'Eastern Visayas (VIII)', 
                                                            'Western Visayas (VI)'])]

# Insert Data to MySQL DB

abstracted `connection.json` 

In [9]:
connection = open('connection.json')
connection = json.load(connection)
host = connection['host']
user = connection['user']
password = connection['password']
port0 = connection['port0']
port1 = connection['port1']
port2 = connection['port2']

db_name0 = 'center'
engine0 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port0)

db_name1 = 'node_2'
engine1 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port1)

db_name2 = 'node_3'
engine2 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port2)

Define the schemas of CSV files

In [10]:
# appointments schema
appointments_schema = {
    "pxid":db.types.VARCHAR(32),
    "clinicid":db.types.VARCHAR(32),
    "doctorid":db.types.VARCHAR(32),
    "apptid":db.types.VARCHAR(32),
    "status":db.types.Enum("Queued", "Complete", "Serving","Cancel","NoShow","Skip"),
    "TimeQueued":db.types.DATETIME(),
    "QueueDate":db.types.DATE(),
    "StartTime":db.types.DATETIME(),
    "EndTime":db.types.DATETIME(),
    "type":db.types.Enum("Consultation","Inpatient"),
    "Virtual":db.types.BOOLEAN(),
    "hospitalname":db.types.VARCHAR(255),
    "IsHospital":db.types.BOOLEAN(),
    "City":db.types.VARCHAR(255),
    "Province":db.types.VARCHAR(255),
    "RegionName":db.types.VARCHAR(255),
    "mainspecialty":db.types.VARCHAR(255),
    "patientage":db.dialects.mysql.TINYINT(255),
    "patientgender":db.types.Enum("MALE", "FEMALE")
}

Insert the merge_df to central node

In [11]:
# central node
with engine0.connect() as conn:
    conn.execute(db.text("DROP DATABASE IF EXISTS `" + db_name0 + "`;"))
    conn.execute(db.text("CREATE DATABASE `" + db_name0 + "`;"))
engine0 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port0 + '/' + db_name0)

In [12]:
merge_df.to_sql('appointments', engine0, if_exists='replace', index=False, dtype=appointments_schema)

320140

In [13]:
with engine0.connect() as conn:
    conn.execute(db.text("""
        ALTER TABLE `""" + db_name0 + """`.`appointments` 
        CHANGE COLUMN `apptid` `apptid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`apptid`);
    """))
    print("apptid IS A PRIMARY KEY IN CENTRAL NODE")

apptid IS A PRIMARY KEY IN CENTRAL NODE


Insert the luzon_df to node 2

In [14]:
# node 2
with engine1.connect() as conn:
    conn.execute(db.text("DROP DATABASE IF EXISTS `" + db_name1 + "`;"))
    conn.execute(db.text("CREATE DATABASE `" + db_name1 + "`;"))
engine1 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port1 + '/' + db_name1)

In [15]:
luzon_df.to_sql('appointments', engine1, if_exists='replace', index=False, dtype=appointments_schema)

261600

In [16]:
with engine1.connect() as conn:
    conn.execute(db.text("""
        ALTER TABLE `""" + db_name1 + """`.`appointments` 
        CHANGE COLUMN `apptid` `apptid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`apptid`);
    """))
    print("apptid IS A PRIMARY KEY IN NODE 2")

apptid IS A PRIMARY KEY IN NODE 2


Insert the visayas_mindanao_df to node 3

In [17]:
# node 3
with engine2.connect() as conn:
    conn.execute(db.text("DROP DATABASE IF EXISTS `" + db_name2 + "`;"))
    conn.execute(db.text("CREATE DATABASE `" + db_name2 + "`;"))
engine2 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port2 + '/' + db_name2)

In [18]:
visayas_mindanao_df.to_sql('appointments', engine2, if_exists='replace', index=False, dtype=appointments_schema)

58540

In [19]:
with engine2.connect() as conn:
    conn.execute(db.text("""
        ALTER TABLE `""" + db_name2 + """`.`appointments` 
        CHANGE COLUMN `apptid` `apptid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`apptid`);
    """))
    print("apptid IS A PRIMARY KEY IN NODE 3")

apptid IS A PRIMARY KEY IN NODE 3


# Setup Logs in MySQL DB

## Variable Setup

In [124]:
connection = open('connection.json')
connection = json.load(connection)
host = connection['host']
user = connection['user']
password = connection['password']
port0 = connection['port0']
port1 = connection['port1']
port2 = connection['port2']

db_name0 = 'center'
engine0 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port0 + '/' + db_name0)

db_name1 = 'node_2'
engine1 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port1 + '/' + db_name1)

db_name2 = 'node_3'
engine2 = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port2 + '/' + db_name2)

# LOGS TABLE
drop_logs = db.text("DROP TABLE IF EXISTS logs;")

def create_logs(centered):
    if not centered:
        auto_inc = "AUTO_INCREMENT"
    else:
        auto_inc = ""
        
    return db.text(f"""
        CREATE TABLE logs (
            `id` INT {auto_inc},
            `node` enum('node_2', 'node_3'),
            `action_time` TIMESTAMP NOT NULL,
            `action` varchar(255),
            `apptid` varchar(32) NOT NULL,
            `status` enum('Queued','Complete','Serving','Cancel','NoShow','Skip') DEFAULT NULL,
            `TimeQueued` datetime DEFAULT NULL,
            `QueueDate` date DEFAULT NULL,
            `StartTime` datetime DEFAULT NULL,
            `EndTime` datetime DEFAULT NULL,
            `type` enum('Consultation','Inpatient') DEFAULT NULL,
            `Virtual` tinyint(1) DEFAULT NULL,
            `hospitalname` varchar(255) DEFAULT NULL,
            `IsHospital` tinyint(1) DEFAULT NULL,
            `City` varchar(255) DEFAULT NULL,
            `Province` varchar(255) DEFAULT NULL,
            `RegionName` varchar(255) DEFAULT NULL,
            `mainspecialty` varchar(255) DEFAULT NULL,
            `patientage` tinyint DEFAULT NULL,
            `patientgender` enum('MALE','FEMALE') DEFAULT NULL,
            PRIMARY KEY (`id`,`node`)
        );
    """)

# TRIGGERS
def drop_triggers(db_name,q):
    return db.text(f"DROP TRIGGER IF EXISTS {db_name}.{q.lower()}_data;")

def create_triggers(q, centered):
    if q.upper() == 'DELETE':
        old_new = 'OLD'
    else:
        old_new = 'NEW'
        
    node_selection = f"""
        SELECT CASE 
            WHEN {old_new}.RegionName = 'Western Visayas (VI)'
              OR {old_new}.RegionName = 'Central Visayas (VII)'
              OR {old_new}.RegionName = 'Eastern Visayas (VIII)'
              OR {old_new}.RegionName = 'Northern Mindanao (X)'
              OR {old_new}.RegionName = 'SOCCSKSARGEN (Cotabato Region) (XII)'
            THEN 'node_3'
            ELSE 'node_2'
        END
    """
    values = f"""NOW(), '{q.upper()}', node_get, {old_new}.apptid, {old_new}.status, {old_new}.TimeQueued, {old_new}.QueueDate, 
        {old_new}.StartTime, {old_new}.EndTime, {old_new}.type, {old_new}.Virtual, {old_new}.hospitalname, {old_new}.IsHospital, {old_new}.City, 
        {old_new}.Province, {old_new}.RegionName, {old_new}.mainspecialty, {old_new}.patientage, {old_new}.patientgender"""
    if not centered: 
        return db.text(f"""
                CREATE TRIGGER {q.lower()}_data AFTER {q.upper()} ON appointments
                FOR EACH ROW
                BEGIN
                    DECLARE node_get VARCHAR(255);
                    SET node_get = ({node_selection});
                    INSERT INTO logs (`action_time`, `action`, `node`, `apptid`, `status`, `TimeQueued`, `QueueDate`, `StartTime`, `EndTime`, `type`, `Virtual`, `hospitalname`, `IsHospital`, `City`, `Province`, `RegionName`, `mainspecialty`, `patientage`, `patientgender`)
                    VALUES({values});
                END;
            """)
    else:
        return db.text(f"""
                CREATE TRIGGER {q.lower()}_data AFTER {q.upper()} ON appointments
                FOR EACH ROW
                BEGIN
                    DECLARE id_get INT;
                    DECLARE node_get VARCHAR(255);
                    SET node_get = ({node_selection});
                    SET id_get = (
                        SELECT MAX(id) FROM logs WHERE node = node_get
                    );
                    SET id_get = (SELECT CASE
                        WHEN id_get IS NULL
                        THEN 0
                        ELSE id_get
                      END);
                    INSERT INTO logs (`id`, `action_time`, `action`, `node`, `apptid`, `status`, `TimeQueued`, `QueueDate`, `StartTime`, `EndTime`, `type`, `Virtual`, `hospitalname`, `IsHospital`, `City`, `Province`, `RegionName`, `mainspecialty`, `patientage`, `patientgender`)
                    VALUES(id_get + 1, {values});
                END;
            """)

## Create Logs Table


### Central Node

In [125]:
with engine0.connect() as conn:
    conn.execute(drop_logs)
    conn.execute(create_logs(True))
    print("logs TABLE CREATED IN Central NODE")

logs TABLE CREATED IN Central NODE


### Luzon Node

In [126]:
with engine1.connect() as conn:
    conn.execute(drop_logs)
    conn.execute(create_logs(False))
    print("logs TABLE CREATED IN Luzon NODE")

logs TABLE CREATED IN Luzon NODE


### VisMin Node

In [127]:
with engine2.connect() as conn:
    conn.execute(drop_logs)
    conn.execute(create_logs(False))
    print("logs TABLE CREATED IN VisMin NODE")

logs TABLE CREATED IN VisMin NODE


## Triggers

### Central Node

In [128]:
with engine0.connect() as conn:
    conn.execute(drop_triggers('center', 'insert'))
    conn.execute(drop_triggers('center', 'update'))
    conn.execute(drop_triggers('center', 'delete'))
    conn.execute(create_triggers('insert', True))
    conn.execute(create_triggers('update', True))
    conn.execute(create_triggers('delete', True))
    print("TRIGGERS CREATED IN Central NODE")

TRIGGERS CREATED IN Central NODE


### Node 2 (Luzon)

In [129]:
with engine1.connect() as conn:
    conn.execute(drop_triggers('node_2', 'insert'))
    conn.execute(drop_triggers('node_2', 'update'))
    conn.execute(drop_triggers('node_2', 'delete'))
    conn.execute(create_triggers('insert', False))
    conn.execute(create_triggers('update', False))
    conn.execute(create_triggers('delete', False))
    print("TRIGGERS CREATED IN Luzon NODE")

TRIGGERS CREATED IN Luzon NODE


### Node 3 (Visayas Mindanao)

In [130]:
with engine2.connect() as conn:
    conn.execute(drop_triggers('node_3', 'insert'))
    conn.execute(drop_triggers('node_3', 'update'))
    conn.execute(drop_triggers('node_3', 'delete'))
    conn.execute(create_triggers('insert', False))
    conn.execute(create_triggers('update', False))
    conn.execute(create_triggers('delete', False))
    print("TRIGGERS CREATED IN Luzon NODE")

TRIGGERS CREATED IN Luzon NODE
